In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import base64


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# update with your username and password and CRUD Python module name
username = "aacuser"
password = "jamjam"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
data = shelter.read_all()
df = pd.DataFrame.from_records(data, columns=['age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                              'date_of_birth', 'name', 'outcome_subtype',
                                              'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long'])

water = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                     "breed":{"$in":["Chesapeake Bay Retriever",
                                                                     "Newfoundland",
                                                                     "Labrador Retriever Mix"]},
                                                     "sex_upon_outcome":"Intact Female",
                                                     "age_upon_outcome_in_weeks":{"$gte":26},
                                                     "age_upon_outcome_in_weeks":{"$lte":156}}))

mountain = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                     "breed":{"$in":["Alaskan Malamute",
                                                                     "German Shepherd",
                                                                     "Old English Sheepdog",
                                                                     "Siberian Husky",
                                                                     "Rottweiler"]},
                                                     "sex_upon_outcome":"Intact Male",
                                                     "age_upon_outcome_in_weeks":{"$gte":26},
                                                     "age_upon_outcome_in_weeks":{"$lte":156}}))
disaster = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                     "breed":{"$in":["Doberman Pinscher",
                                                                     "German Shepherd",
                                                                     "Golden Retriever",
                                                                     "Bloodhound",
                                                                     "Rottweiler"]},
                                                     "sex_upon_outcome":"Intact Male",
                                                     "age_upon_outcome_in_weeks":{"$gte":20},
                                                     "age_upon_outcome_in_weeks":{"$lte":300}}))

# Print statements to ensure each filtering option is working correctly

#print(mountain) 
#print(disaster)
#print(water)


#########################
# Dashboard Layout / View
#########################
# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.A(href="https://www.snhu.edu", children=html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                                                          style={'width': '100px', 'height': '100px'})),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.H1("Timothy McIntyre"),
    html.Hr(),
    # Buttons for animal by type
    html.Div(className='row',
             style={'display':'flex'},
             children =[
                 html.Button('Water Rescue',id="button-one", n_clicks=0),
                 html.Button('Disaster Rescue',id="button-two", n_clicks=0),
                 html.Button('Mountain Rescue',id="button-three", n_clicks=0),
                 html.Button('Reset',id="button-four", n_clicks=0),
             ]),
    html.Hr(),

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False, 
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        style_table={'height':'400px','overflowY':'auto','overflowX':'auto'},
        style_header={'backgroundColor':'rgb(240,230,230)',
                     'fontWeight':'bold'},
        style_data={'whiteSpace':'normal','height':'auto'},
         #tooltips 
        tooltip ={i: {'value': i,'use_with': 'both'} for i in df.columns},
        tooltip_delay=0,
        tooltip_duration = None,
        row_selectable = 'single',
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your geolocation chart and Graph are side by side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id', 'columns'), # I have tried with and without this
              ],
             [Input('button-one','n_clicks'),# Water
             Input('button-two','n_clicks'),# Disaster
             Input('button-three','n_clicks'),# Mountain
             Input('button-four','n_clicks'),])# Reset

def update_dashboard(btn1='button-one', btn2='button-two', btn3='button-three', btn4='button-four'):
    if (int(btn1) >= 1):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                     "breed":{"$in":["Chesapeake Bay Retriever",
                                                                     "Newfoundland",
                                                                     "Labrador Retriever Mix"]},
                                                     "sex_upon_outcome":"Intact Female",
                                                     "age_upon_outcome_in_weeks":{"$gte":26},
                                                     "age_upon_outcome_in_weeks":{"$lte":156}}))
        btn2, btn3, btn4 = 0,0,0
    elif (int(btn2) >= 1):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                     "breed":{"$in":["Doberman Pinscher",
                                                                     "German Shepherd",
                                                                     "Golden Retriever",
                                                                     "Bloodhound",
                                                                     "Rottweiler"]},
                                                     "sex_upon_outcome":"Intact Male",
                                                     "age_upon_outcome_in_weeks":{"$gte":20},
                                                     "age_upon_outcome_in_weeks":{"$lte":300}}))
        btn1, btn3, btn4 = 0,0,0
    elif (int(btn3) >= 1):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                     "breed":{"$in":["Alaskan Malamute",
                                                                     "German Shepherd",
                                                                     "Old English Sheepdog",
                                                                     "Siberian Husky",
                                                                     "Rottweiler"]},
                                                     "sex_upon_outcome":"Intact Male",
                                                     "age_upon_outcome_in_weeks":{"$gte":26},
                                                     "age_upon_outcome_in_weeks":{"$lte":156}}))
        btn1, btn2, btn4 = 0,0,0
    elif (int(btn4) >= 1):
        df = pd.DataFrame.from_records(shelter.read_all())
        btn1, btn2, btn3 = 0,0,0
    
    columns = [{"name": i,"id": i, "deletable":False, "selectable":True} for i in df.columns]
    data = df.to_dict('records')
    
    
    return data

# Style 
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'),
    Input('datatable-id', 'selected_rows')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Pie graph of animal types

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(view_data):
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(view_data)
    breeds = []
    for i in range(0, dff.__len__()):
        breeds.append(str(dff.iloc[i, 3]))
    return [
        dcc.Graph(
            figure={
                'data': [
                    {
                        'labels': breeds,
                        'type': 'pie'
                    }
                ],
                'layout': {
                    'title': 'Breeds',
                }
            }
        )
    ]


# map with market of first row of chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(view_data):
    if view_data is None:
        return []
    else:
        # Add in the code for your geolocation chart
        dff = pd.DataFrame.from_dict(view_data)
        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '500px', 'height': '250px'}, center=[dff.iloc[0, 10], dff.iloc[0, 11]], zoom=10,
                   children=[
                       dl.TileLayer(id="base-layer-id"),
                       # Marker with tool tip and popup
                       dl.Marker(position=[dff.iloc[0, 10], dff.iloc[0, 11]], children=[
                           dl.Tooltip(dff.iloc[0, 6]),
                           dl.Popup([
                               html.H1(dff.iloc[0, 3]),
                               html.H1(dff.iloc[0, 2]),
                               html.P(dff.iloc[0, 0] + " old")
                           ])
                       ])
                   ])
        ]



app